In [11]:
#Converter .WAV para .wav
import os
import glob
import shutil

def converter_todos_wav_em_pasta(pasta):
    """
    Converte .WAV para .wav - versão robusta para Windows
    """
    if not os.path.exists(pasta):
        print("❌ Pasta não encontrada!")
        return 0
    
    # Encontra arquivos .WAV (case sensitive)
    arquivos_wav = []
    for arquivo in os.listdir(pasta):
        if arquivo.endswith('.WAV'):
            caminho_completo = os.path.join(pasta, arquivo)
            arquivos_wav.append(caminho_completo)
    
    if not arquivos_wav:
        print("❌ Nenhum arquivo .WAV encontrado.")
        return 0
    
    print(f"📁 Pasta: {pasta}")
    print(f"🔍 Encontrados {len(arquivos_wav)} arquivo(s) .WAV")
    print("-" * 50)
    
    convertidos = 0
    
    for arquivo_original in arquivos_wav:
        nome_arquivo = os.path.basename(arquivo_original)
        novo_nome = nome_arquivo[:-4] + '.wav'
        novo_caminho = os.path.join(pasta, novo_nome)
        
        print(f"🔄 Processando: {nome_arquivo}")
        
        # Verifica se realmente é um arquivo diferente
        if nome_arquivo.lower() == novo_nome.lower():
            print(f"   ⚠️  Mesmo arquivo (Windows case-insensitive). Renomeando...")
        
        try:
            # Tenta o rename direto primeiro
            os.rename(arquivo_original, novo_caminho)
            print(f"   ✅ Convertido para: {novo_nome}")
            convertidos += 1
            
        except OSError as e:
            # Se der erro, usa cópia + remoção
            print(f"   ⚠️  Método alternativo necessário...")
            try:
                shutil.copy2(arquivo_original, novo_caminho)
                os.remove(arquivo_original)
                print(f"   ✅ Convertido (cópia) para: {novo_nome}")
                convertidos += 1
            except Exception as e2:
                print(f"   ❌ Falha na conversão: {e2}")
    
    print("-" * 50)
    print(f"🎉 Conversão concluída: {convertidos}/{len(arquivos_wav)} arquivos convertidos")
    
    # Verificação final
    print("\n🔍 Verificando resultado...")
    arquivos_convertidos = [f for f in os.listdir(pasta) if f.endswith('.wav')]
    print(f"📊 Arquivos .wav na pasta agora: {len(arquivos_convertidos)}")
    
    return convertidos

if __name__ == "__main__":
    pasta = r"C:\Users\Ricardo\Desktop\Áudios treinamento\Gravador 09"
    converter_todos_wav_em_pasta(pasta)

📁 Pasta: C:\Users\Ricardo\Desktop\Áudios treinamento\Gravador 09
🔍 Encontrados 60 arquivo(s) .WAV
--------------------------------------------------
🔄 Processando: 20230605_054800.WAV
   ⚠️  Mesmo arquivo (Windows case-insensitive). Renomeando...
   ✅ Convertido para: 20230605_054800.wav
🔄 Processando: 20230605_055800.WAV
   ⚠️  Mesmo arquivo (Windows case-insensitive). Renomeando...
   ✅ Convertido para: 20230605_055800.wav
🔄 Processando: 20230607_060000.WAV
   ⚠️  Mesmo arquivo (Windows case-insensitive). Renomeando...
   ✅ Convertido para: 20230607_060000.wav
🔄 Processando: 20230607_145800.WAV
   ⚠️  Mesmo arquivo (Windows case-insensitive). Renomeando...
   ✅ Convertido para: 20230607_145800.wav
🔄 Processando: 20230619_102400.WAV
   ⚠️  Mesmo arquivo (Windows case-insensitive). Renomeando...
   ✅ Convertido para: 20230619_102400.wav
🔄 Processando: 20230619_102600.WAV
   ⚠️  Mesmo arquivo (Windows case-insensitive). Renomeando...
   ✅ Convertido para: 20230619_102600.wav
🔄 Processan

In [12]:
#Trocar a classificação dos meus rótulos para o do programa de pedro
import os
import pandas as pd
import glob
import re

def processar_arquivos_vocalizacoes(pasta_origem):
    """
    Processa arquivos TXT com dados de vocalizações, mantendo apenas as 4 classificações
    especificadas e convertendo para códigos abreviados.
    """
    
    mapeamento_classificacao = {
        'phee adulto': 'p',
        'trill adulto': 't', 
        'phee bb': 'e',
        'trill bb': 'l'
    }
    
    # Usar pasta processados (não criar se já existir)
    pasta_processados = os.path.join(pasta_origem, "processados")
    
    padrao_arquivos = os.path.join(pasta_origem, '*.txt')
    arquivos_txt = glob.glob(padrao_arquivos)
    
    if not arquivos_txt:
        print(f"Nenhum arquivo TXT encontrado na pasta: {pasta_origem}")
        return
    
    print(f"Encontrados {len(arquivos_txt)} arquivo(s) TXT para processar.")
    print(f"Arquivos processados serão salvos em: {pasta_processados}")
    
    for caminho_arquivo in arquivos_txt:
        try:
            nome_arquivo = os.path.basename(caminho_arquivo)
            caminho_destino = os.path.join(pasta_processados, nome_arquivo)
            
            # Ler e processar o arquivo
            with open(caminho_arquivo, 'r', encoding='utf-8') as f:
                linhas = f.readlines()
            
            dados_processados = []
            linhas_ignoradas = 0
            
            for linha in linhas:
                linha = linha.strip()
                if not linha:  # Pular linhas vazias
                    continue
                
                # Dividir por qualquer espaço em branco
                partes = re.split(r'\s+', linha)
                
                if len(partes) >= 3:
                    # Juntar todas as partes além da 2ª para formar a classificação completa
                    inicio = partes[0]
                    fim = partes[1]
                    classificacao = ' '.join(partes[2:]).lower()
                    
                    # Verificar se é uma das classificações que queremos manter
                    if classificacao in mapeamento_classificacao:
                        codigo = mapeamento_classificacao[classificacao]
                        dados_processados.append([inicio, fim, codigo])
                    else:
                        linhas_ignoradas += 1
                else:
                    linhas_ignoradas += 1
            
            # Salvar o arquivo processado COM cabeçalho usando vírgula como separador
            with open(caminho_destino, 'w', encoding='utf-8') as f:
                # Escrever cabeçalho com vírgula
                f.write("onset_s,offset_s,label\n")
                
                # Escrever dados com vírgula como separador
                for inicio, fim, classificacao in dados_processados:
                    linha_formatada = f"{inicio},{fim},{classificacao}\n"
                    f.write(linha_formatada)
            
            # Estatísticas
            total_linhas_originais = len([l for l in linhas if l.strip()])
            total_linhas_processadas = len(dados_processados)
            
            print(f"Processado: {nome_arquivo}")
            print(f"  Linhas originais: {total_linhas_originais}")
            print(f"  Linhas mantidas: {total_linhas_processadas}")
            print(f"  Linhas ignoradas: {linhas_ignoradas}")
            
            if linhas_ignoradas > 0:
                print(f"  (Classificações não reconhecidas foram removidas)")
            
        except Exception as e:
            print(f"Erro ao processar {nome_arquivo}: {str(e)}")

def mostrar_classificacoes_arquivos(pasta_origem):
    """
    Mostra todas as classificações encontradas nos arquivos para referência
    """
    print("\n=== CLASSIFICAÇÕES ENCONTRADAS NOS ARQUIVOS ===")
    
    arquivos_txt = glob.glob(os.path.join(pasta_origem, '*.txt'))
    todas_classificacoes = set()
    
    for caminho_arquivo in arquivos_txt[:5]:  # Verifica apenas os primeiros 5 arquivos
        try:
            with open(caminho_arquivo, 'r', encoding='utf-8') as f:
                for linha in f:
                    partes = linha.strip().split()
                    if len(partes) >= 3:
                        classificacao = ' '.join(partes[2:]).lower()
                        todas_classificacoes.add(classificacao)
        except:
            pass
    
    print("Classificações encontradas:")
    for cls in sorted(todas_classificacoes):
        status = "✓ Mapeada" if cls in ['phee adulto', 'trill adulto', 'phee bb', 'trill bb'] else "✗ Ignorada"
        print(f"  {cls} -> {status}")

if __name__ == "__main__":
    caminho_pasta = input("Digite o caminho completo da pasta com os arquivos TXT: ")
    
    if not os.path.exists(caminho_pasta):
        print("A pasta especificada não existe!")
    else:
        # Primeiro mostra quais classificações existem
        mostrar_classificacoes_arquivos(caminho_pasta)
        
        # Verificar se pasta processados existe
        pasta_processados = os.path.join(caminho_pasta, "processados")
        if not os.path.exists(pasta_processados):
            print(f"\nAviso: A pasta 'processados' não existe em {caminho_pasta}")
            print("O processamento será cancelado.")
        else:
            # Pergunta se quer continuar
            print("\nDeseja continuar com o processamento?")
            resposta = input("Pressione ENTER para SIM ou ESPAÇO + ENTER para NÃO: ")
            
            if resposta.strip() == "":  # Enter sem digitar nada = SIM
                processar_arquivos_vocalizacoes(caminho_pasta)
                print("\nProcessamento concluído!")
                print("Os arquivos processados foram salvos na pasta 'processados'")
                print("Os arquivos incluem cabeçalho e estão separados por VÍRGULA:")
                print("  - onset_s")
                print("  - offset_s") 
                print("  - label")
            else:
                print("Processamento cancelado.")


=== CLASSIFICAÇÕES ENCONTRADAS NOS ARQUIVOS ===
Classificações encontradas:
  phee adulto -> ✓ Mapeada
  phee bb -> ✓ Mapeada
  trill adulto -> ✓ Mapeada
  trill bb -> ✓ Mapeada

Deseja continuar com o processamento?
Encontrados 60 arquivo(s) TXT para processar.
Arquivos processados serão salvos em: C:\Users\Ricardo\Desktop\Áudios treinamento\Gravador 09\processados
Processado: 20230605_054800.txt
  Linhas originais: 1
  Linhas mantidas: 1
  Linhas ignoradas: 0
Processado: 20230605_055800.txt
  Linhas originais: 10
  Linhas mantidas: 10
  Linhas ignoradas: 0
Processado: 20230607_060000.txt
  Linhas originais: 10
  Linhas mantidas: 10
  Linhas ignoradas: 0
Processado: 20230607_145800.txt
  Linhas originais: 1
  Linhas mantidas: 1
  Linhas ignoradas: 0
Processado: 20230619_102400.txt
  Linhas originais: 4
  Linhas mantidas: 4
  Linhas ignoradas: 0
Processado: 20230619_102600.txt
  Linhas originais: 6
  Linhas mantidas: 6
  Linhas ignoradas: 0
Processado: 20230619_102800.txt
  Linhas ori

In [16]:
import os
import pandas as pd
import glob

def converter_txt_para_csv_pasta_separada(pasta_base):
    """
    Converte arquivos TXT da pasta 'processados' para CSV com formato: nome.wav.csv
    """
    
    pasta_origem = os.path.join(pasta_base, "processados")
    pasta_destino = os.path.join(pasta_base, "csv")
    
    if not os.path.exists(pasta_origem):
        print(f"❌ Pasta 'processados' não encontrada em: {pasta_base}")
        return False
    
    # Criar pasta destino se não existir
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)
        print(f"📁 Pasta 'csv' criada: {pasta_destino}")
    
    # Encontrar arquivos TXT
    arquivos_txt = glob.glob(os.path.join(pasta_origem, '*.txt'))
    
    if not arquivos_txt:
        print(f"❌ Nenhum arquivo TXT encontrado em: {pasta_origem}")
        return False
    
    print(f"📄 Arquivos TXT encontrados: {len(arquivos_txt)}")
    print(f"📍 Origem: {pasta_origem}")
    print(f"🎯 Destino: {pasta_destino}")
    
    arquivos_convertidos = 0
    
    for caminho_txt in arquivos_txt:
        try:
            nome_arquivo = os.path.basename(caminho_txt)
            
            # CORREÇÃO: Remover .txt e adicionar .wav.csv
            # Exemplo: arquivo.txt → arquivo.wav.csv
            nome_base = os.path.splitext(nome_arquivo)[0]  # Remove .txt
            nome_csv = nome_base + '.wav.csv'
            caminho_csv = os.path.join(pasta_destino, nome_csv)
            
            # Ler TXT e converter para CSV
            df = pd.read_csv(caminho_txt, sep=',', encoding='utf-8')
            df.to_csv(caminho_csv, index=False, encoding='utf-8')
            
            print(f"✅ {nome_arquivo} → {nome_csv} ({len(df)} linhas)")
            arquivos_convertidos += 1
            
        except Exception as e:
            print(f"❌ Erro em {nome_arquivo}: {e}")
    
    print(f"\n🎉 Conversão concluída!")
    print(f"📊 Arquivos convertidos: {arquivos_convertidos}/{len(arquivos_txt)}")
    print(f"📁 Arquivos CSV salvos em: {pasta_destino}")
    
    return True

def main():
    """
    Programa principal - converte para pasta separada.
    """
    print("CONVERSOR TXT → CSV (FORMATO: nome.wav.csv)")
    print("=" * 60)
    
    caminho_base = input("Digite o caminho onde está a pasta 'processados': ").strip('"')
    
    if not os.path.exists(caminho_base):
        print("❌ Caminho não existe!")
        return
    
    pasta_processados = os.path.join(caminho_base, "processados")
    
    if not os.path.exists(pasta_processados):
        print("❌ Pasta 'processados' não encontrada!")
        return
    
    # Mostrar arquivos que serão convertidos
    arquivos_txt = glob.glob(os.path.join(pasta_processados, '*.txt'))
    print(f"\n📁 Arquivos encontrados na pasta 'processados':")
    for arquivo in arquivos_txt:
        nome = os.path.basename(arquivo)
        print(f"  📄 {nome}")
    
    print(f"\n📝 Formato dos nomes CSV:")
    print(f"   exemplo.txt → exemplo.wav.csv")
    print(f"📍 Pasta destino: {caminho_base}\\csv")
    
    # Lógica de confirmação corrigida
    print("\n✅ Deseja continuar?")
    resposta = input("Pressione ENTER para SIM ou digite qualquer coisa + ENTER para NÃO: ")
    
    if resposta == "":  # Enter sem digitar nada = SIM
        print("\n🔄 Convertendo...")
        converter_txt_para_csv_pasta_separada(caminho_base)
    else:
        print("❌ Conversão cancelada.")

if __name__ == "__main__":
    main()

CONVERSOR TXT → CSV (FORMATO: nome.wav.csv)

📁 Arquivos encontrados na pasta 'processados':
  📄 20220921_124200.txt
  📄 20220921_135000.txt
  📄 20221003_090400.txt
  📄 20221003_090600.txt
  📄 20221003_090800.txt
  📄 20221003_091800.txt
  📄 20221003_092600.txt
  📄 20221003_094400.txt
  📄 20221003_100600.txt
  📄 20221003_101600.txt
  📄 20221003_102000.txt
  📄 20221003_103200.txt
  📄 20221007_090400.txt
  📄 20221007_091400.txt
  📄 20221007_091600.txt
  📄 20221007_092000.txt
  📄 20221007_092200.txt
  📄 20221007_092400.txt
  📄 20221007_093600.txt
  📄 20221007_093800.txt
  📄 20221007_095200.txt
  📄 20221007_102000.txt
  📄 20221007_102400.txt
  📄 20221024_102000.txt
  📄 20221024_102200.txt
  📄 20221024_102400.txt
  📄 20221024_102600.txt
  📄 20221028_100200.txt
  📄 20221028_101200.txt
  📄 20221028_104000.txt
  📄 20221028_104400.txt
  📄 20221109_090400.txt
  📄 20221109_090600.txt
  📄 20221109_093000.txt
  📄 20221109_093600.txt
  📄 20221109_144000.txt
  📄 20221109_144600.txt
  📄 20221128_085200.